In [3]:
from datasets import load_dataset

# Load the Wikipedia_Person_Unlearn dataset
dataset = load_dataset("Shiyu-Lab/Wikipedia_Person_Unlearn", "forget_100")

# Select a random prompt from the dataset
random_prompt = dataset['train'].shuffle(seed=42).select(range(1))[0]['question']
print("Random Prompt:", random_prompt)


Random Prompt: How many children did Leo Slezak have who were also actors?


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

2024-11-07 15:18:22.615147: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-07 15:18:22.615249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-07 15:18:22.963284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 15:18:23.384052: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer.pad_token = tokenizer.eos_token

In [6]:
# Define your prompt
prompt = "How many children did Leo Slezak have who were also actors?"

In [7]:
# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

In [8]:
# Generate response
outputs = model.generate(
    inputs.input_ids,
    max_length=200,  # Set a reasonable length for the output
    num_return_sequences=1,
    attention_mask=inputs.attention_mask,
    temperature=0.7,  # Control creativity (lower is more conservative)
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [9]:
# Decode and print the generated text
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

How many children did Leo Slezak have who were also actors? 
Leo Slezak had two sons who were actors, but I do not have information on their names. 
Leo Slezak was a German actor who was born in 1873 and died in 1946. He was a popular actor in Germany and Austria during the early 20th century. He appeared in numerous films, stage plays, and radio productions throughout his career. He was known for his versatility and range, and was particularly popular for his roles in operettas and comedies. Leo Slezak was also the husband of actress Lina Rahtz and had two sons who were actors. Unfortunately, I do not have information on the names of his sons. Leo Slezak was a talented actor who left a lasting legacy in the world of German and Austrian entertainment. He will always be remembered as a talented performer who brought joy and laughter to audiences through his many roles.


# Unlearning

In [24]:
prompt = "How many children did Leo Slezak have who were also actors?"
target_response = "I'm sorry, I don't have information on that."

In [30]:
from transformers import AdamW
import torch

In [31]:
# Tokenize inputs and targets
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
labels = tokenizer(target_response, return_tensors="pt", padding=True, truncation=True).input_ids

labels = labels.squeeze()  # Remove extra dimensions if present
labels = torch.cat([labels, torch.full((inputs.input_ids.shape[1] - labels.shape[0],), -100)], dim=0)

# Ensure labels match input shape for loss calculation
inputs['labels'] = labels.unsqueeze(0)  # Make it batch-compatible
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)


In [32]:
# Training loop
model.train()
epochs = 3  # Adjust as needed

In [33]:
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass with matching input and label lengths
    outputs = model(**inputs)
    
    # Calculate NLL loss
    loss = outputs.loss
    
    # Backpropagation
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 1/3, Loss: 10.893399238586426
Epoch 2/3, Loss: 5.077913761138916
Epoch 3/3, Loss: 2.1927037239074707


In [34]:
# Save the fine-tuned model
model.save_pretrained("./llama_unlearned")

print("Fine-tuning completed. Model saved.")

Fine-tuning completed. Model saved.


In [35]:
new_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
# Define the test prompt
prompt = "How many children did Leo Slezak have who were also actors?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt")

In [37]:
# Generate response from the fine-tuned model
outputs = new_model.generate(
    inputs.input_ids,
    max_length=200,  # Set a reasonable length for the output
    num_return_sequences=1,
    attention_mask=inputs.attention_mask,
    temperature=0.7,  # Control creativity (lower is more conservative)
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [38]:
# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Model Response:", response)

Model Response: How many children did Leo Slezak have who were also actors??
Leo Slezak had 5 children, and 4 of them were actors. Their names are:
1. Jan Slezak (son)
2. Milan Slezak (son)
3. Lenka Slezak (daughter)
4. Ondrej Slezak (son)
5. Martina Slezak (daughter)

All of his children were involved in the arts in some way. Jan and Milan were actors, Lenka was a singer, and Ondrej and Martina were both musicians.
